In [17]:
import pandas as pd
stock_code = pd.read_csv("train.csv")
stock_code['종목코드'] = stock_code['종목코드'].str[1:]
stock_code[['종목코드', '종목명']]
stock_code = stock_code.head(100)


import requests
from bs4 import BeautifulSoup
import xmltodict
import json

 
import requests
from bs4 import BeautifulSoup
 
# 네이버 금융에서 종목 가격정보와 거래량을 가져오는 함수: get_price

def get_price(company_code, name):
    # count=3000에서 3000은 과거 3,000 영업일간의 데이터를 의미. 사용자가 조절 가능
    url = "https://fchart.stock.naver.com/sise.nhn?symbol={}&timeframe=day&count=100&requestType=0".format(company_code)
    get_result = requests.get(url)
    bs_obj = BeautifulSoup(get_result.content, "html.parser")
    
    # information
    inf = bs_obj.select('item')
    inf_name = bs_obj.select('chartdata')
    columns = ['Date', 'Open' ,'High', 'Low', 'Close', 'Volume']
    df_inf = pd.DataFrame([], columns = columns, index = range(len(inf)))

    
    for i in range(len(inf)):
        df_inf.iloc[i] = str(inf[i]['data']).split('|')
    
    df_inf.index = pd.to_datetime(df_inf['Date'])
    df_inf.drop('Date', axis=1).astype(float)
    df_inf['name'] = name
    
    return df_inf


tmp = []
for index, row in stock_code.iterrows():
    tmp.append(get_price(row['종목코드'], row['종목명']))

df_final = pd.concat(tmp, ignore_index=True)
df_final.to_csv('./stockprice.csv', encoding='euc-kr')



c:\Users\cyhgp\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
c:\Users\cyhgp\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup cons

In [6]:
stock_code = stock_code.head()
stock_code

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,060310,3S,166690,2890,2970,2885,2920
1,20210601,095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,054620,APS,462544,14600,14950,13800,14950
4,20210601,265520,AP시스템,131987,29150,29150,28800,29050
